# Feature Engineer de Atributos Linear Regression

## *Senário Proposto*

- Conseguir o melhor R2 Score (Testando modelo Linear)
- Usar apenas Regressão Linear Simples
- Usar Feature Engineer!

### Librarys

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

- Dados

In [ ]:
 
df = pd.read_csv('/content/visits.csv', sep='\t')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

- 

In [ ]:
 
df['day_of_week'] = df['date'].dt.day_of_week
df['month'] = df['date'].dt.month
df['day_of_month'] = df['date'].dt.day
 

- Bolean - Dia de pagameto => True

In [ ]:
df['is_payday'] = df['day_of_month'].isin([1, 2, 3, 4, 5, 28, 29, 30, 31]).astype(int)

OQ FAZ

In [ ]:

 
df['day_month_inter'] = df['day_of_week'].astype(str) + '_' + df['month'].astype(str)
df = pd.concat([df, pd.get_dummies(df['day_month_inter'], prefix='inter', drop_first=True, dtype=float)], axis=1)
 

In [ ]:
 

feriados = [
    '2023-01-01', '2023-02-20', '2023-02-21', '2023-02-22',
    '2023-04-07', '2023-04-21', '2023-05-01', '2023-06-08',
    '2023-09-07', '2023-10-12', '2023-11-02', '2023-11-15', '2023-12-25'
]
df['is_holiday'] = df['date'].isin(pd.to_datetime(feriados)).astype(int)

In [ ]:
for i in range(1, 15):
    df[f'lag_{i}'] = df['visits'].shift(i)
 
df['roll_mean_7'] = df['visits'].shift(1).rolling(window=7).mean()
df['roll_max_7'] = df['visits'].shift(1).rolling(window=7).max()
df['roll_min_7'] = df['visits'].shift(1).rolling(window=7).min()

In [ ]:

outliers_extremos = [
    '2023-11-14', '2023-06-10', '2023-08-08', '2023-04-09', '2023-08-13', '2023-05-14',
    '2023-08-15', '2023-04-28', '2023-02-18', '2023-08-11', '2023-08-26'
]
 
for date in outliers_extremos:
    col_name = f'special_{date}'
    df[col_name] = (df['date'] == pd.to_datetime(date)).astype(int)
 
df_model = df.dropna().copy()
y = df_model['visits']
cols_to_drop = ['date', 'visits', 'day_of_week', 'month', 'day_of_month', 'day_month_inter']
X = df_model.drop(cols_to_drop, axis=1)
X = X.astype(float)
 


In [ ]:
model = LinearRegression()
model.fit(X, y)
score = model.score(X, y)
 
print("-" * 30)
print(f"R² (Scikit-Learn): {score:.4f}")
print("-" * 30)
 
X_sm = sm.add_constant(X)
model_sm = sm.OLS(y, X_sm).fit()
 
print(model_sm.summary().tables[0])